# Load the Model

In [ ]:
import open_clip

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
model.eval()

tokenizer = open_clip.get_tokenizer('ViT-B-32')

tokenizer, model

# Get Emoji Embeddings

In [ ]:
from PIL import Image

from IPython.display import display

import torch

def get_embedding(emoji_name):
    image = preprocess(Image.open(f'{emoji_name}.png')).unsqueeze(0)
    text = tokenizer([emoji_name])
    
    with torch.no_grad(), torch.autocast("cuda"):
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    
        emoji_embedding = 0.5*image_features + 0.5*text_features
        emoji_embedding /= emoji_embedding.norm(dim=-1, keepdim=True)
    
        emoji_embedding.cpu()

    return emoji_embedding

emojis = [
    'kekleo',
    'fatcat'
]

E = torch.cat([
    get_embedding(emoji) for emoji in emojis
])

E

# Get Text Query Embedding

In [ ]:
def get_text_embedding(emoji_name):
    text = tokenizer([emoji_name])
    
    with torch.no_grad(), torch.autocast("cuda"):
        text_embedding = model.encode_text(text)
        text_embedding /= text_embedding.norm(dim=-1, keepdim=True)
        text_embedding.cpu()

    return text_embedding

query_embedding = get_text_embedding('cat')

query_embedding.shape

# Compute Scores

In [ ]:
scores = E @ query_embedding.T

scores

# Get the Best Scoring Emoji

In [ ]:
max_score_idx = torch.argmax(scores, dim=0).item()  # idx = 1

max_emoji = emojis[max_score_idx]

print(max_emoji)

Image.open(f'{max_emoji}.png')